In [16]:
import pandas as pd
import numpy as np
import geopandas as gpd
import shapely
from pykml import parser
# from lxml import etree
import mercantile
import os
import re
import requests
from PIL import Image

In [214]:
with open("/mnt/d/dev/situated-views/data-in/kml-sessions/001ALA011001.kml", "r+") as f:
    kml_str = f.read()
    parsed = parser.fromstring(kml_str)
    parsed.PhotoOverlay.Camera.altitude = 500
    str = etree.tostring(parsed, pretty_print=True)
    f.seek(0)
    f.write(str.decode("utf-8"))
    f.truncate()
    #with open("/mnt/d/dev/situated-views/data-in/test.kml", "w") as k:
    #    k.write(str.decode("utf-8"))

In [8]:
def find_with_re(property, string):
    return re.search(f"(?<=<{property}>).+(?=<\/{property}>)", string).group(0)

In [50]:
def split_photooverlays(kml_folder, delete_original=False):
    with open(kml_folder, "r") as f:
        txt = f.read()
        header = "\n".join(txt.split("\n")[:2])
        photooverlays = re.split(".(?=<PhotoOverlay>)", txt)[1:]
    for po in photooverlays:
        filename = find_with_re("name", po)
        with open(os.path.join(os.path.dirname(kml_folder), filename + ".kml"), "w") as k:
            k.write(f"{header}\n{po}</kml>")
    if delete_original:
        os.remove(os.path.abspath(kml_folder))

In [238]:
split_photooverlays("/mnt/d/dev/situated-views/data-in/kml-sessions/2019-04-12-Davi.kml")

In [5]:
def change_img_href(kml):
    with open(kml, "r+") as f:
        txt = f.read()
        filename = find_with_re("name", txt)
        txt = re.sub(
        "(?<=<href>).+(?=<\/href>\n\t+<\/Icon>\n\t+<ViewVolume>)",
        f"https://rioiconography.sfo2.digitaloceanspaces.com/situatedviews/{filename}.jpg",
        txt
        )
        f.seek(0)
        f.write(txt)
        f.truncate()

In [19]:
def correct_altitude_mode():
    with open("/mnt/d/dev/situated-views/data-in/kml-sessions/001ALA011001.kml", "r+") as f:
        txt = f.read()
        if re.search("(?<=<altitudeMode>)relative(?=<\/altitudeMode>)", txt):
            lat = float(find_with_re("latitude", txt))
            lon = float(find_with_re("longitude", txt))
            alt = float(find_with_re("altitude", txt))
        z = 15
        x = mercantile.tile(lon, lat, z).x
        y = mercantile.tile(lon, lat, z).y
        rgb_tile = requests.get(f"https://api.mapbox.com/v4/mapbox.terrain-rgb/{z}/{x}/{y}.pngraw?access_token=pk.eyJ1IjoibWFydGltcGFzc29zIiwiYSI6ImNra3pmN2QxajBiYWUycW55N3E1dG1tcTEifQ.JFKSI85oP7M2gbeUTaUfQQ")
        array = Image.open(rgb_tile.content)
        print(array)
        return array

In [20]:
correct_altitude_mode()

UnboundLocalError: local variable 'lon' referenced before assignment